In [2]:
import boto3
import pprint
import json
import uuid
import logging
import pandas as pd
from datetime import datetime

print(boto3.__version__)
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

region = "us-east-1"
client = boto3.client("bedrock-agent-runtime", region_name=region)
logger.info(client)

[2023-12-14 09:32:21,315] p112533 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


[2023-12-14 09:32:21,387] p112533 {2105213047.py:15} INFO - <botocore.client.AgentsforBedrockRuntime object at 0x7fb60fa6b160>


1.33.11


In [ ]:
# "promptOverrideConfiguration": { 
#     "overrideLambda": "string",
#     "promptConfigurations": [ 
#         { 
#             "basePromptTemplate": "string",
#             "inferenceConfiguration": { 
#                 "maximumLength": int,
#                 "stopSequences": [ "string" ],
#                 "temperature": float,
#                 "topK": float,
#                 "topP": float
#             },
#             "parserMode": "DEFAULT | OVERRIDDEN",
#             "promptCreationMode": "DEFAULT | OVERRIDDEN",
#             "promptState": "ENABLED | DISABLED",
#             "promptType": "PRE_PROCESSING | ORCHESTRATION | KNOWLEDGE_BASE_RESPONSE_GENERATION | POST_PROCESSING"
#         }
#     ]
# }


# response = client.update_agent(
#     agentId='KDZQM7M2ZN',
#     agentName='kingdee-agent-test',
#     promptOverrideConfiguration={
#         'promptConfigurations': [
#             {
#                 'promptType': 'PRE_PROCESSING',
#                 'promptCreationMode': 'DEFAULT'|'OVERRIDDEN',
#                 'promptState': 'ENABLED'|'DISABLED',
#                 'basePromptTemplate': 'string',
#                 'inferenceConfiguration': {
#                     'temperature': ...,
#                     'topP': ...,
#                     'topK': 123,
#                     'maximumLength': 123,
#                     'stopSequences': [
#                         'string',
#                     ]
#                 },
#                 'parserMode': 'DEFAULT'|'OVERRIDDEN'
#             },
#         ],
#         'overrideLambda': 'string'
#     }
# )

In [97]:
response = client.update_agent(
    agentId='KDZQM7M2ZN',
    agentName='kingdee-agent-test',
    foundationModel='anthropic.claude-v2',
    AgentsforBedrockRuntime="1800",
    promptOverrideConfiguration={
        'promptConfigurations': [
            {
                'promptType': 'PRE_PROCESSING',
                'promptState': 'DISABLED',
            },
        ],
    },
    agentResourceRoleArn = "arn:aws:iam::596899493901:role/service-role/AmazonBedrockExecutionRoleForAgents_47OQ8P7NB76"
)

AttributeError: 'AgentsforBedrockRuntime' object has no attribute 'update_agent'

In [5]:
def invoke(question: str, sessionid: str, agent_id: str, agent_alias_id: str, enable_trace=False):
    response = client.invoke_agent(inputText=question,
        agentId=agent_id,
        agentAliasId=agent_alias_id,
        sessionId=sessionid,
        enableTrace=enable_trace
    )
    pre_process_time = 0
    orchestration_time = 0
    post_process_time = 0
    total_time = 0
    final_answer = ""
    event_stream = response['completion']
    now  = datetime.now()
    try:
        for event in event_stream:
            duration_time = (datetime.now()-now).seconds
            print(f"duration_time: {duration_time}")
            print(event)
            if 'chunk' in event:
                data = event['chunk']['bytes']
                # print()
                # logger.info(f"Final answer ->\n{data.decode('utf8')}") 
                final_answer = data.decode('utf8')
                print(f"Final answer ->\n{final_answer}") 
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                trace = json.dumps(event['trace'], indent=2)
                if "preProcessingTrace" in trace:
                    pre_process_time += duration_time 
                elif "orchestrationTrace" in trace:
                    orchestration_time += duration_time
                elif "postProcessingTrace" in trace:
                    post_process_time += duration_time 
                # divmod((duration_time).total_seconds(), 60)
                # logger.info(trace)
                # pprint.pprint(trace)
            else:
                raise Exception("unexpected event.", event)
            now  = datetime.now()
    except Exception as e:
        raise Exception("unexpected event.", e)
    print(f"pre_process_time: {pre_process_time}")
    print(f"orchestration_time: {orchestration_time}")
    print(f"post_processtime: {post_process_time}")
    total_time = pre_process_time + orchestration_time + post_process_time
    pre_process_time_final = f"{pre_process_time/total_time*100:.1f}% ({pre_process_time})"
    orchestration_time_final = f"{orchestration_time/total_time*100:.1f}% ({orchestration_time})"
    post_process_time_final = f"{post_process_time/total_time*100:.1f}% ({post_process_time})"
    return pre_process_time_final, orchestration_time_final, post_process_time_final, total_time, final_answer 
    # if enable_trace:
    #     return trace
    # else:
    #     return None

In [86]:
sessionid = str(uuid.uuid1())
agent_alias_id = "JZI7ACBPHL"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
# time_pd = pd.DataFrame(data=None,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
lst = []
# pre_process_time_list
# orchestration_time_list = 
# post_process_time_list = 
# total_time_list = 
# question_list = 
# output_list = 

In [87]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])
# time_pd = time_pd.append(dict(zip(['pre process time','orchestration time','post process time','total time', "input", "output"], [pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])), ignore_index=True)
# print(time_pd)

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'c47df0ec-9a08-481c-ae93-c538f7165c87-pre-0', 'text': "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requi

In [88]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'c31ef312-b12c-4a3f-b023-be0791e32fab-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [89]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'bade6490-3a3d-427e-871d-66fe5b7c8540-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [90]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'ee6cb3a3-9ccc-4fd9-8cd1-0dad91322cca-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [91]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'c38968a2-e892-4f8a-b4f6-a248f14977fa-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [126]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'd94a3eea-f52e-40d8-aa37-646d654f8f8b-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [93]:
question = "please sent to my email address"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': '544b4312-971a-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '3f7ac24b-95a6-45a1-a473-3b5de7103735-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [81]:
pre_process_time_format = f"{pre_process_time/total_time*100:.1f}% ({pre_process_time})"
pre_process_time_format

'29.6% (8)'

In [69]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'JZI7ACBPHL', 'sessionId': 'a10ed2fa-9714-11ee-bda6-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '41de127e-dd39-4a83-9ff0-eafe112ece95-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [94]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test.xlsx")

In [3]:
sessionid = str(uuid.uuid1())
agent_alias_name = "no-preprocessing"
agent_alias_id = "VYUXKYCBXX"
agent_id = "KDZQM7M2ZN"
enable_trace:bool = True 

In [13]:
pprint.pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1131',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 01 Dec 2023 07:12:58 GMT',
                                      'x-amzn-requestid': 'ae206424-22f3-48ac-9a43-0ce3f793466a'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'ae206424-22f3-48ac-9a43-0ce3f793466a',
                      'RetryAttempts': 0},
 'citations': [{'generatedResponsePart': {'textResponsePart': {'span': {'end': 26,
                                                                        'start': 0},
                                                               'text': 'Jun '
                                                                       'Deng负责amazon '
                                                                       'sagemaker。'}},
                'retrievedReference

In [5]:
question = "hi"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
Can you please provide your company name, tax number, user id, product name, product code, and product price?


In [6]:
question = "我要开发票"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
I apologize, I do not have enough information yet to generate an invoice. Could you please provide the buyer company name, buyer tax number, your user id, product name, product code, and product price?


In [7]:
question = "91440300MA5FAE9E4P"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
Thank you for providing the tax number. Could you also please provide the buyer company name, your user id, product name, product code, and product price so I can generate a preview of the invoice?


In [8]:
question = "金蝶有礼"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
Thank you for the company name. Could you please also provide your user id, the product name, product code, and price so that I can generate a preview of the invoice?


In [9]:
question = "00001，小麦，1010101020000000000，9000"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
Here is a preview of the invoice: <REDACTED>{"snapShotUrl":"https://api-sit.piaozone.com/doc/free/fileInfo/preview/s11813099252574904321265218486"}<REDACTED>

Please review it and let me know if you would like me to proceed with generating the formal invoice file.


In [10]:
question = "确认"
invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

Final answer ->
I'm sorry, there seems to be an issue with generating the invoice file right now. I tried to generate the formal invoice but received an error indicating the invoice tool is failing. Please try again later and I will attempt to generate the invoice again.


## 2. Disable pre-process

In [120]:
sessionid = str(uuid.uuid1())
agent_alias_name = "disable-pre"
agent_alias_id = "PONE3PFOKP"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [121]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '0f5b959e-5ba1-48b6-ae1e-e01c708f715d-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [122]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'abb263d7-1723-4fab-96c8-53dfcdc672f0-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [123]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'e799e875-809d-43e2-b956-6d2b03fbb2a4-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 8
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': 'e799e875-809d-43e2-b956-6d2b03fbb2a4-0', 'text': 'The user has not provided all the required parameters. I will ask again for the missing information.'}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'observation': {'traceId': 'e799e875-809d-43e2-b956-6d2b03fbb2a4-0', 'type': 'ASK_USER', 'finalResponse': {'text': 'Can you please also provide your user id and product details (name, code, price for each product)?'}}}}}}
duration_time: 0
{'chunk': {'bytes': b'Can you please also provide your user id and product details (name, code, price for each product)?'}}
Final answer ->
Can you please also provide your user id and product details (name, code, price for each product)?
p

In [124]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'cb1ccd52-047d-427c-8eff-98a124b0b8b6-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [125]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '5a91f8b1-d5cb-4ca2-9427-bad8f6d190ba-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [127]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'c601ae4a-2818-413b-921c-2a9314d96b7b-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 12
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': 'c601ae4a-2818-413b-921c-2a9314d96b7b-0', 'text': 'Based on reviewing the conversation history, the user had previously:\n\n1. Provided invoice details (buyer tax number, buyer company name, user ID, product details)\n\n2. I generated a preview invoice image using those details \n\n3. Returned the text information from the preview image \n\n4. Asked the user to confirm if they want to proceed with generating the actual invoice\n\nThe user has now responded with "yes", indicating they want me to proceed with generating the actual invoice.\n\nTo continue assisting the user:\n\n1. I will call GET::InvocieService::issueInvoice to generate the actual invoice, using the parameters previously provided by the user\n\n2. If successful, I will return the downloadUrl so the user can download the invoice\n

In [128]:
question = "please sent to my email address"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '70074d6e-80f7-4449-b281-5bff120c198f-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [129]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PONE3PFOKP', 'sessionId': 'da755dbc-9731-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '33e20b64-43c2-4313-be55-f48001cf7094-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [130]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test_disable_pre.xlsx")

## 3. Orchestration prompt 测试

In [131]:
sessionid = str(uuid.uuid1())
agent_alias_name = "disable-pre-rm2eample"
agent_alias_id = "NCQYKX0FXE"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
# time_pd = pd.DataFrame(data=None,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
lst = []


In [132]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '68291083-1916-4378-996a-de06db4e6e66-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 8
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': '68291083-1916-4378-996a-de06db4e6e66-0', 'text': 'Since the user simply greeted me, I will use a friendly greeting in response based on my instructions.'}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'observation': {'traceId': '68291083-1916-4378-996a-de06db4e6e66-0', 'type': 'FINISH', 'finalResponse': {'text': "Hello! I'm an invoice assistant."}}}}}}
duration_time: 0
{'chunk': {'bytes': b"Hello! I'm an invoice assistant."}}
Final answer ->
Hello! I'm an invoice assistant.
pre_process_time: 0
orchestration_time: 8
post_processtime: 0


In [133]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '91aa20f6-66ca-43fa-ab73-e6c2007dbc9d-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [134]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'aaf523ff-e0e6-40cf-aa39-18a44cef9687-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 6
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': 'aaf523ff-e0e6-40cf-aa39-18a44cef9687-0', 'text': 'The user has not provided all the required information. I will ask again for the missing details.'}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'observation': {'traceId': 'aaf523ff-e0e6-40cf-aa39-18a44cef9687-0', 'type': 'ASK_USER', 'finalResponse': {'text': 'Can you please also provide your user id and product details?'}}}}}}
duration_time: 0
{'chunk': {'bytes': b'Can you please also provide your user id and product details?'}}
Final answer ->
Can you please also provide your user id and product details?
pre_process_time: 0
orchestration_time: 6
post_processtime: 0


In [135]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '265779d0-91ed-410c-90b4-31f787fbafb5-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [136]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'e85b4470-16fa-4ada-a1cd-a4514e2dc386-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 9
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': 'e85b4470-16fa-4ada-a1cd-a4514e2dc386-0', 'text': 'I now have all the required invoice information. I will call generatePreviewInvoiceImage to generate a preview image.'}}}}}
duration_time: 1
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'invocationInput': {'traceId': 'e85b4470-16fa-4ada-a1cd-a4514e2dc386-0', 'invocationType': 'ACTION_GROUP', 'actionGroupInvocationInput': {'actionGroupName': 'InvocieService', 'verb': 'post', 'apiPath': '/generatePreviewInvoiceImage', 'parameters': [{'name': 'buyer_company_name', 'type': 'string', 'value': 'test_company_name'}, {'name': 'product_detail', 'type': 'array', 'value': '["{"name":"小麦","code":"1010101020000000000","money":"9000"}"]'}, {'name': 'user

In [137]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '0f91e6b8-5e54-454b-9e9f-4e6af8ed4e74-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [138]:
question = "please sent to my email address"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'c21b5b28-91bf-4ee5-a634-5332721cc40a-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [139]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'bffb416f-99ca-443a-9b50-7b3ab89f2653-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 11
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': 'bffb416f-99ca-443a-9b50-7b3ab89f2653-0', 'text': "I now have the user's email address. Next I need to ask them for the invoice code and invoice number."}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NCQYKX0FXE', 'sessionId': 'a371a88a-9735-11ee-bda6-0efac58ff0bf', 'trace': {'orchestrationTrace': {'observation': {'traceId': 'bffb416f-99ca-443a-9b50-7b3ab89f2653-0', 'type': 'ASK_USER', 'finalResponse': {'text': 'Can you please provide the invoice code and invoice number?'}}}}}}
duration_time: 0
{'chunk': {'bytes': b'Can you please provide the invoice code and invoice number?'}}
Final answer ->
Can you please provide the invoice code and invoice number?
pre_process_time: 0
orchestration_time: 11
post_processtime: 0


In [140]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test_disable_pre_reduce_example.xlsx")

## 4. disabel pre-process, remove 2 example, reduce funtion, repalce ask

In [4]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask"
agent_alias_id = "NHEDALYYDD"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []


In [5]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NHEDALYYDD', 'sessionId': '3cfbb5fc-9a53-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '1ec6d157-bd06-4e78-a768-1fc8d39f4f5e-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a frien

In [6]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NHEDALYYDD', 'sessionId': '3cfbb5fc-9a53-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'dc75bb4e-fa24-4339-9cb3-7a76eb585914-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a frien

In [7]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NHEDALYYDD', 'sessionId': '3cfbb5fc-9a53-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'e61125df-9700-4247-b88c-6dafce94ec9a-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a frien

In [8]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'NHEDALYYDD', 'sessionId': '3cfbb5fc-9a53-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'b40c1e0e-1ff0-4a90-a760-89e2b1371ba3-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a frien

## 5. disabel pre-process, remove 2 example, reduce funtion, repalce ask v2

In [9]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask2"
agent_alias_id = "HZ8YBSKDV3"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [10]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'd7cd05ec-923c-4b1b-9311-80c481a9d44a-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [11]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'eb52879b-f2aa-42cb-8e9f-92d87495ea48-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [12]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '247a74e5-2a0c-420e-9f04-046ad88c83e3-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [13]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '663a9a54-3596-4e2b-a8c8-d3e4e95a8953-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 6
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': '663a9a54-3596-4e2b-a8c8-d3e4e95a8953-0', 'text': 'I still do not have the complete product details. I will ask the user again.'}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'observation': {'traceId': '663a9a54-3596-4e2b-a8c8-d3e4e95a8953-0', 'type': 'ASK_USER', 'finalResponse': {'text': 'Can you please provide the product details including name, code, and price for each product?'}}}}}}
duration_time: 0
{'chunk': {'bytes': b'Can you please provide the product details including name, code, and price for each product?'}}
Final answer ->
Can you please provide the product details including name, code, and price for each product?
pre_process_time: 0
orchestration_time: 6
p

In [14]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '9d5f75c0-a2fe-477c-b515-9a02b0eae7e3-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [15]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '1b9b4d6e-ea3b-458a-931d-4c20437ec20a-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [16]:
question = "please sent to my email address"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'e3fcc084-3d32-46a1-9c5d-0f5d2a904a43-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [18]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HZ8YBSKDV3', 'sessionId': '7b70f4c2-9a54-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '94905cf0-d954-4c82-9306-e80d77020f19-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [19]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test_disable_pre_reduce_example_rdfunc_and_fixask_bug.xlsx")

## 6. disabel pre-process, remove 2 example, reduce funtion, repalce ask v2 rmmulti_ex

In [20]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask2_rmmulti_ex"
agent_alias_id = "HFNJZVLKYQ"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [21]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '556ad8a0-b167-45a9-9850-e595508bc122-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [22]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '6aaacd27-846f-4600-90c0-bc9a526d3ec6-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [23]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'd381de38-7b9b-4086-948d-636426948102-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [24]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'f421999d-4dec-4bd7-9bb8-e5df672620e8-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [25]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '73710e18-6647-4312-833d-9b101b5edc1a-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [26]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'e5a1796f-f3de-43c1-92f0-313af3e38af7-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [27]:
question = "please sent to my email address"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '3331e898-aeb8-42f1-be65-a076d7895914-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

duration_time: 10
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'rationale': {'traceId': '3331e898-aeb8-42f1-be65-a076d7895914-0', 'text': "Based on the conversation history, the user has requested an invoice be generated and I was able to successfully generate it using the provided functions. \n\nThe user's latest question is asking for the invoice to be sent to their email address. To do this, I need the user's email address and the invoice details like invoice code and invoice number.\n\nI can use the user::askuser function to ask the user for their email address. \n\nI already have the invoice code and invoice number from the previous function results when generating the invoice.\n\nOnce I have the email address, I can call the GET::InvocieService::sendInvoiceEmail function to email the invoice."}}}}}
duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVL

In [28]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'HFNJZVLKYQ', 'sessionId': 'd7c1f024-9a58-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'bcd69532-9502-4f25-8b26-6a9fa038a017-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [29]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test_disable_pre_reduce_example_rdfunc_and_fixask_bug_rmmultiex.xlsx")

## 7. disabel pre-process, remove 2 example, reduce funtion, repalce ,sk v2 rmmulti_ex, reduce more funcs in example

In [45]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask2_rmmulti_ex_more"
agent_alias_id = "PQN0DD83RQ"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [46]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PQN0DD83RQ', 'sessionId': '876fec36-9a60-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': '5a73282b-5ca1-46d3-8f7f-a622d36507b3-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [47]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PQN0DD83RQ', 'sessionId': '876fec36-9a60-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'b0f78cc0-db86-46f5-9be3-533455464681-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [48]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PQN0DD83RQ', 'sessionId': '876fec36-9a60-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'd77f0458-8050-4059-b46b-145f2118e874-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

In [49]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'PQN0DD83RQ', 'sessionId': '876fec36-9a60-11ee-beaf-0efac58ff0bf', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'traceId': 'bd74a5cb-0eaf-4c51-a7c1-cfa6225d62d1-0', 'text': '\n\nHuman:\nYou are a research assistant AI that has been equipped with one or more functions to help you answer a <question>. Your goal is to answer the user\'s question to the best of your ability, using the function(s) to gather more information if necessary to better answer the question. If you choose to call a function, the result of the function call will be added to the conversation history in <function_results> tags (if the call succeeded) or <error> tags (if the function failed). If you need to call a function but have not yet been provided with all the relevant parameters, you may ask the user for these missing parameters.\n\nYou were created with these instructions to consider as well:\n<auxiliary_instructions>You are a fri

## 8. disabel pre-process, remove 2 example, reduce funtion, repalce ,sk v2 rmmulti_ex, instant

In [3]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask2_rmmulti_ex_instant"
agent_alias_id = "U57EGWWO4P"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [6]:
question = "hi"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '8823099e-e8d0-4e05-aa4b-9191d1d0baeb-pre-0', 'text': "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requi

In [7]:
question = "I want to issue an invoice "
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'e999ffbc-8af6-497f-a645-dcb63346f937-pre-0', 'text': "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requi

In [8]:
question = "91440300MA5FAE9E4P, test_company_name"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'b73e5425-2581-4039-b662-a11bc34e112a-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [9]:
question = "00001"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '8c59fb34-bdea-42fd-8dd1-838ae9c157b3-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [10]:
question = "小麦，1010101020000000000，9000"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'a3d4c757-0b46-4334-88d7-9400c45d432e-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [11]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '3056606a-f7d7-4d99-9769-dca776abd653-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [12]:
question = "yes"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '1694fa4a-0419-4772-8b2d-7fcb78549110-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [13]:
question = "xiaoqunn@amazon.com"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': 'b354ed80-9a63-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': 'a8f4e419-978f-4312-9a6c-d6cd8bdfe1e8-pre-0', 'text': '\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user\'s questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requ

In [14]:
time_pd = pd.DataFrame(data=lst,columns=['pre process time','orchestration time','post process time','total time', "input", "output"])
time_pd.to_excel("test_disable_pre_reduce_example_rdfunc_and_fixask_bug_rmmultiex_instant.xlsx")

In [21]:
sessionid = str(uuid.uuid1())
agent_alias_name = "dispre_rm2ex_rmfunc_repask2_rmmulti_ex_instant"
agent_alias_id = "U57EGWWO4P"
agent_id = "FP1NSLWFHD"
enable_trace:bool = True
lst = []

In [22]:
question = "您好"
pre_process_time, orchestration_time, post_process_time, total_time, final_answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)
lst.append([pre_process_time, orchestration_time, post_process_time, total_time, question, final_answer])

duration_time: 0
{'trace': {'agentId': 'FP1NSLWFHD', 'agentAliasId': 'U57EGWWO4P', 'sessionId': '315b0ab0-9a7e-11ee-9a19-0efac58ff0bf', 'trace': {'preProcessingTrace': {'modelInvocationInput': {'traceId': '29ad35b2-bf71-4b7b-ba53-33b669b979c4-pre-0', 'text': "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvocieService::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_number (string): The tax number of buyer company</required_argument>\n<requi